In [28]:
import datetime
print(f'{datetime.datetime.now()} Analysis Setup')

import sklearn as sk
import anndata as ad
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os

os.chdir('/project/hipaa_ycheng11lab/atlas/CAMR2024')
sc.settings.n_jobs = -1

adata = ad.read_h5ad('data/camr_scrublet_batch_filtered.h5ad', backed = 'r')

raw = True
which_set = "Curated" #"Curated", "Coeff-0.3"
data_string = "NORMALIZED_COUNTS"
if raw:
    data_string = "RAW_COUNTS"

2024-08-18 12:53:38.363581 Analysis Setup


In [39]:
merged_filtered_markers.loc[merged_filtered_markers["Major_Name"] == "AC"]

,Name,Marker,Curated,Queried,Minor_Coefficient,Major_Name,Major_Coefficient,Queried_Name,Raw_Mean_Expression_Subtype_Marker,Raw_Mean_Expression_Majorclass_Marker,...,BC,Cone,Endothelial,HC,MG,Microglia,Pericyte,RGC,RPE,Rod
0,AC1,Rbfox1,NaN,Queried,0.561532,AC,NaN,AC_1,9.738955,5.628321,...,0.082111,0.338376,0.02253,0.293651,0.076488,0.13856,0.091811,13.777289,0.005128,0.077222
1,AC1,Stmn1,NaN,Queried,0.414927,AC,NaN,AC_1,6.531861,4.484476,...,0.838333,0.11613,0.665511,1.34127,2.318807,0.82002,0.126551,21.688866,0.192308,0.079088
2,AC10,Kcnip4,NaN,Queried,0.479949,AC,NaN,AC_10,15.004365,7.037623,...,14.641668,0.02003,0.10052,1.690476,2.444016,0.165187,0.094293,43.91038,0.051282,0.037293
3,AC10,Nrxn1,NaN,Queried,0.339957,AC,NaN,AC_10,6.302778,4.514096,...,2.208109,0.083529,0.071057,9.753968,0.066204,0.049474,0.096774,8.582273,0.005128,0.014603
4,AC14,Frmd5,NaN,Queried,0.386736,AC,NaN,AC_14,9.316994,6.016793,...,0.477381,1.898146,1.745234,2.214286,1.613832,0.035996,0.114144,0.427972,0.023077,0.665936
5,AC19,Cntnap2,NaN,Queried,0.301338,AC,NaN,AC_19,13.923743,5.444735,...,0.667272,0.017473,0.062392,0.293651,1.241805,0.101414,0.171216,9.863459,0.266667,0.014633
6,AC2,C1ql1,NaN,Queried,0.424057,AC,NaN,AC_2,7.385806,5.018295,...,0.001379,0.006606,0.019931,8.888889,0.095256,0.123603,0.064516,0.135381,0.015385,0.008857
7,AC2,Ptprd,NaN,Queried,0.695098,AC,NaN,AC_2,18.411507,5.550095,...,12.524196,0.020882,0.195841,3.333333,8.913742,0.060158,0.327543,12.522441,0.023077,0.023519
8,AC2,Rbfox1,NaN,Queried,0.380587,AC,NaN,AC_2,12.593166,5.628321,...,0.082111,0.338376,0.02253,0.293651,0.076488,0.13856,0.091811,13.777289,0.005128,0.077222
9,AC20,Pcdh9,NaN,Queried,0.655432,AC,NaN,AC_20,25.521297,4.411223,...,23.634071,0.670147,0.436742,20.738094,13.283712,0.06213,0.272953,11.684127,0.069231,0.046712


In [3]:
# Clean subtypes for plotting
adata.obs = adata.obs.loc[:, ["majorclass", "author_cell_type"]]
adata.obs["author_cell_type"] = adata.obs["author_cell_type"].astype(str)
is_unassigned = adata.obs["author_cell_type"] == adata.obs["majorclass"]
unassigned_subtypes = adata.obs["author_cell_type"].loc[is_unassigned]
adata.obs.loc[is_unassigned, "author_cell_type"] = ["Unassigned_" + uv for uv in unassigned_subtypes]

adata.var = adata.var.loc[:, ["gene_symbols", "feature_name", "feature_length"]]
adata.var["feature_name"] = adata.var["feature_name"].astype(str).str.capitalize()
adata.var["feature_length"] = adata.var["feature_length"].astype(int)
adata.var.index = adata.var["feature_name"] # subset on genes instead of booleans
adata.var_names = adata.var["feature_name"] # subset on genes instead of booleans

In [4]:
# if not os.path.isfile('spreadsheets/spreadsheets/filtered_curated_ByCell_markers_with_length_rawcounts_Coefficients-0.3.txt'):
merged_filtered_markers = pd.read_csv('spreadsheets/merged_curated-queried_ByCell_markers_with_Coefficients.txt', sep = '\t')
small_coef = np.logical_or(merged_filtered_markers["Minor_Coefficient"] <= 0.3, merged_filtered_markers["Major_Coefficient"] <= 0.3)
small_coef_in_query = np.logical_and(merged_filtered_markers["Queried"] == "Queried", small_coef)
merged_filtered_markers = merged_filtered_markers[~small_coef_in_query]
# merged_filtered_markers.to_csv('spreadsheets/filtered_merged_ByCell_markers_with_Coefficients.txt', sep ='\t', index = False)
# if which_set == "Curated":
#     merged_filtered_markers = merged_filtered_markers[np.logical_or(merged_filtered_markers["Curated"] == "Curated", merged_filtered_markers["Queried"] == "Queried")]
merged_filtered_markers

,Name,Marker,Curated,Queried,Minor_Coefficient,Major_Name,Major_Coefficient,Queried_Name
0,AC,Anks1b,NaN,Queried,NaN,AC,0.450539,AC
1,AC,Asic2,NaN,Queried,NaN,AC,0.561311,AC
2,AC,C1ql1,NaN,Queried,NaN,AC,0.441690,AC
3,AC,Cd90,Curated,NaN,NaN,AC,NaN,NaN
4,AC,Dlgap1,NaN,Queried,NaN,AC,0.343129,AC
...,...,...,...,...,...,...,...,...
2470,ROD,Pde6a,Curated,NaN,NaN,ROD,NaN,NaN
2472,ROD,Rho,NaN,Queried,NaN,ROD,0.538664,Rod
2474,ROD,Sag,NaN,Queried,NaN,ROD,0.348941,Rod
2481,RPE,Rlbp1,Curated,NaN,NaN,RPE,NaN,NaN


In [ ]:
# raw counts
## subclass
major_sub_mean = pd.DataFrame(columns = ['Queried_Name','Marker','Raw_Mean_Expression_Subtype_Marker'])
for major_sub_class in ["AC", "BC", "Microglia", "RGC"]:
    subcell_raw_mean = pd.read_csv(f'data/raw_{major_sub_class}_subtype_meanExpression.csv')
    subcell_raw_mean.loc[subcell_raw_mean["author_cell_type"] == major_sub_class, "author_cell_type"] = "Unassigned_" + major_sub_class
    subcell_raw_mean_long = pd.melt(subcell_raw_mean, id_vars='author_cell_type', var_name='Marker', value_name='Raw_Mean_Expression_Subtype_Marker')
    subcell_raw_mean_long = subcell_raw_mean_long.rename(columns={'author_cell_type':'Queried_Name'})
    subcell_raw_mean_long["Marker"] = subcell_raw_mean_long["Marker"].str.capitalize()
    merged_filtered_markers.loc[np.logical_and(merged_filtered_markers["Queried_Name"] == major_sub_class, merged_filtered_markers["Minor_Coefficient"] > 0.3)
, "Queried_Name"] = "Unassigned_" + major_sub_class
    major_sub_mean = pd.concat([major_sub_mean, subcell_raw_mean_long], ignore_index = True)
    

In [7]:
major_sub_mean = major_sub_mean.loc[major_sub_mean['Raw_Mean_Expression_Subtype_Marker'] >= 4]

In [8]:
merged_filtered_markers = merged_filtered_markers.merge(major_sub_mean, on=['Queried_Name', 'Marker'])
merged_filtered_markers

,Name,Marker,Curated,Queried,Minor_Coefficient,Major_Name,Major_Coefficient,Queried_Name,Raw_Mean_Expression_Subtype_Marker
0,AC1,Bhlhe22,NaN,Queried,0.750527,AC,NaN,AC_1,4.166313
1,AC1,Fgf14,NaN,Queried,0.413554,AC,NaN,AC_1,5.173109
2,AC1,Lingo2,NaN,Queried,0.597793,AC,NaN,AC_1,7.115973
3,AC1,Ptn,Curated,Queried,0.863408,AC,NaN,AC_1,4.456245
4,AC1,Rbfox1,NaN,Queried,0.561532,AC,NaN,AC_1,9.738955
...,...,...,...,...,...,...,...,...,...
772,W3D2,Pcdh7,NaN,Queried,0.797843,RGC,NaN,23_W3D2,27.031578
773,W3D2,Ptn,NaN,Queried,0.400029,RGC,NaN,23_W3D2,4.474854
774,W3D2,Pvalb,NaN,Queried,0.806446,RGC,NaN,23_W3D2,12.764913
775,W3D2,Rab3c,NaN,Queried,0.408472,RGC,NaN,23_W3D2,4.538012


In [10]:
# majorclass
major_raw_mean = pd.read_csv(f'data/raw_majorclass_meanExpression.csv')
major_raw_mean_long = pd.melt(major_raw_mean, id_vars='majorclass', var_name='Marker', value_name='Raw_Mean_Expression_Majorclass_Marker')
major_raw_mean_long = major_raw_mean_long.rename(columns={'majorclass':'Major_Name'})
major_raw_mean_long["Marker"] = major_raw_mean_long["Marker"].str.capitalize()
major_raw_mean_long = major_raw_mean_long.loc[major_raw_mean_long["Raw_Mean_Expression_Majorclass_Marker"] >= 4]

merged_filtered_markers = merged_filtered_markers.merge(major_raw_mean_long, on=['Major_Name', 'Marker'])

In [11]:
merged_filtered_markers.index = merged_filtered_markers["Marker"]
merged_filtered_markers = merged_filtered_markers.join(adata.var.loc[adata.var["feature_length"] >= 960, "feature_length"])
merged_filtered_markers.index = list(range(merged_filtered_markers.shape[0]))
merged_filtered_markers

,Name,Marker,Curated,Queried,Minor_Coefficient,Major_Name,Major_Coefficient,Queried_Name,Raw_Mean_Expression_Subtype_Marker,Raw_Mean_Expression_Majorclass_Marker,feature_length
0,AC1,Rbfox1,NaN,Queried,0.561532,AC,NaN,AC_1,9.738955,5.628321,4400
1,AC1,Stmn1,NaN,Queried,0.414927,AC,NaN,AC_1,6.531861,4.484476,1922
2,AC10,Kcnip4,NaN,Queried,0.479949,AC,NaN,AC_10,15.004365,7.037623,3627
3,AC10,Nrxn1,NaN,Queried,0.339957,AC,NaN,AC_10,6.302778,4.514096,20733
4,AC14,Frmd5,NaN,Queried,0.386736,AC,NaN,AC_14,9.316994,6.016793,7546
...,...,...,...,...,...,...,...,...,...,...,...
226,W3D2,Camk2d,NaN,Queried,0.392333,RGC,NaN,23_W3D2,8.792982,4.267854,22493
227,W3D2,Kcnd2,NaN,Queried,0.911086,RGC,NaN,23_W3D2,30.659649,5.398889,5152
228,W3D2,Lrrc4c,NaN,Queried,0.515166,RGC,NaN,23_W3D2,17.440935,11.469259,4626
229,W3D2,Pcdh7,NaN,Queried,0.797843,RGC,NaN,23_W3D2,27.031578,4.723438,10604


In [12]:
# all majorclass
major_raw_mean = major_raw_mean.T
major_raw_mean.columns = major_raw_mean.iloc[0]
major_raw_mean["Marker"] = major_raw_mean.index.astype(str).str.capitalize()
major_raw_mean = major_raw_mean.drop(major_raw_mean.index[0])
merged_filtered_markers = merged_filtered_markers.merge(major_raw_mean, on = 'Marker', how = 'left')

merged_filtered_markers

,Name,Marker,Curated,Queried,Minor_Coefficient,Major_Name,Major_Coefficient,Queried_Name,Raw_Mean_Expression_Subtype_Marker,Raw_Mean_Expression_Majorclass_Marker,...,BC,Cone,Endothelial,HC,MG,Microglia,Pericyte,RGC,RPE,Rod
0,AC1,Rbfox1,NaN,Queried,0.561532,AC,NaN,AC_1,9.738955,5.628321,...,0.082111,0.338376,0.02253,0.293651,0.076488,0.13856,0.091811,13.777289,0.005128,0.077222
1,AC1,Stmn1,NaN,Queried,0.414927,AC,NaN,AC_1,6.531861,4.484476,...,0.838333,0.11613,0.665511,1.34127,2.318807,0.82002,0.126551,21.688866,0.192308,0.079088
2,AC10,Kcnip4,NaN,Queried,0.479949,AC,NaN,AC_10,15.004365,7.037623,...,14.641668,0.02003,0.10052,1.690476,2.444016,0.165187,0.094293,43.91038,0.051282,0.037293
3,AC10,Nrxn1,NaN,Queried,0.339957,AC,NaN,AC_10,6.302778,4.514096,...,2.208109,0.083529,0.071057,9.753968,0.066204,0.049474,0.096774,8.582273,0.005128,0.014603
4,AC14,Frmd5,NaN,Queried,0.386736,AC,NaN,AC_14,9.316994,6.016793,...,0.477381,1.898146,1.745234,2.214286,1.613832,0.035996,0.114144,0.427972,0.023077,0.665936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,W3D2,Camk2d,NaN,Queried,0.392333,RGC,NaN,23_W3D2,8.792982,4.267854,...,3.394424,7.856169,1.07539,6.65873,3.024553,1.146121,1.141439,4.267854,0.15641,2.185782
227,W3D2,Kcnd2,NaN,Queried,0.911086,RGC,NaN,23_W3D2,30.659649,5.398889,...,0.520086,0.003835,0.019931,0.079365,0.025582,0.010848,0.079404,5.398889,0.0,0.003673
228,W3D2,Lrrc4c,NaN,Queried,0.515166,RGC,NaN,23_W3D2,17.440935,11.469259,...,4.938026,0.007245,0.215771,1.095238,3.650855,0.046351,1.042184,11.469259,0.007692,0.011996
229,W3D2,Pcdh7,NaN,Queried,0.797843,RGC,NaN,23_W3D2,27.031578,4.723438,...,3.198745,0.005753,0.227903,0.230159,0.933282,0.025148,0.007444,4.723438,0.064103,0.010278


In [14]:
not_crowding = np.sum(merged_filtered_markers.loc[:, 'AC':'Rod'] > 100, axis = 1) == 0
merged_filtered_markers.loc[not_crowding, :]

,Name,Marker,Curated,Queried,Minor_Coefficient,Major_Name,Major_Coefficient,Queried_Name,Raw_Mean_Expression_Subtype_Marker,Raw_Mean_Expression_Majorclass_Marker,...,BC,Cone,Endothelial,HC,MG,Microglia,Pericyte,RGC,RPE,Rod
0,AC1,Rbfox1,NaN,Queried,0.561532,AC,NaN,AC_1,9.738955,5.628321,...,0.082111,0.338376,0.02253,0.293651,0.076488,0.13856,0.091811,13.777289,0.005128,0.077222
1,AC1,Stmn1,NaN,Queried,0.414927,AC,NaN,AC_1,6.531861,4.484476,...,0.838333,0.11613,0.665511,1.34127,2.318807,0.82002,0.126551,21.688866,0.192308,0.079088
2,AC10,Kcnip4,NaN,Queried,0.479949,AC,NaN,AC_10,15.004365,7.037623,...,14.641668,0.02003,0.10052,1.690476,2.444016,0.165187,0.094293,43.91038,0.051282,0.037293
3,AC10,Nrxn1,NaN,Queried,0.339957,AC,NaN,AC_10,6.302778,4.514096,...,2.208109,0.083529,0.071057,9.753968,0.066204,0.049474,0.096774,8.582273,0.005128,0.014603
4,AC14,Frmd5,NaN,Queried,0.386736,AC,NaN,AC_14,9.316994,6.016793,...,0.477381,1.898146,1.745234,2.214286,1.613832,0.035996,0.114144,0.427972,0.023077,0.665936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,W3D2,Camk2d,NaN,Queried,0.392333,RGC,NaN,23_W3D2,8.792982,4.267854,...,3.394424,7.856169,1.07539,6.65873,3.024553,1.146121,1.141439,4.267854,0.15641,2.185782
227,W3D2,Kcnd2,NaN,Queried,0.911086,RGC,NaN,23_W3D2,30.659649,5.398889,...,0.520086,0.003835,0.019931,0.079365,0.025582,0.010848,0.079404,5.398889,0.0,0.003673
228,W3D2,Lrrc4c,NaN,Queried,0.515166,RGC,NaN,23_W3D2,17.440935,11.469259,...,4.938026,0.007245,0.215771,1.095238,3.650855,0.046351,1.042184,11.469259,0.007692,0.011996
229,W3D2,Pcdh7,NaN,Queried,0.797843,RGC,NaN,23_W3D2,27.031578,4.723438,...,3.198745,0.005753,0.227903,0.230159,0.933282,0.025148,0.007444,4.723438,0.064103,0.010278


In [ ]:
merged_filtered_markers.to_csv('spreadsheets/filtered_curated_ByCell_markers_with_length_rawcounts_Coefficients-0.3.txt', sep ='\t', index = False)

In [21]:
long_enough = ~merged_filtered_markers["feature_length"].isnull()
detectable = np.logical_or(~merged_filtered_markers['Raw_Mean_Expression_Subtype_Marker'].isnull(), ~merged_filtered_markers['Raw_Mean_Expression_Majorclass_Marker'].isnull())
not_crowding = np.sum(merged_filtered_markers.loc[:, 'AC':'Rod'] > 100, axis = 1) == 0
sum(np.logical_and(long_enough, np.logical_and(detectable, not_crowding)))

231

In [20]:
merged_filtered_markers["feature_length"]

0       4400
1       1922
2       3627
3      20733
4       7546
       ...  
226    22493
227     5152
228     4626
229    10604
230     1848
Name: feature_length, Length: 231, dtype: int64

In [22]:
merged_filtered_markers.loc[merged_filtered_markers['Marker'] == "Mgarp"]

,Name,Marker,Curated,Queried,Minor_Coefficient,Major_Name,Major_Coefficient,Queried_Name,Raw_Mean_Expression_Subtype_Marker,Raw_Mean_Expression_Majorclass_Marker,...,BC,Cone,Endothelial,HC,MG,Microglia,Pericyte,RGC,RPE,Rod
31,AC4,Mgarp,NaN,Queried,0.322671,AC,NaN,AC_4,14.083260,11.631005,...,17.33726,5.2557,0.311958,35.72222,32.71796,0.668803,0.382134,9.656711,0.328205,1.783412
151,NOVEL_18,Mgarp,NaN,Queried,0.416064,RGC,NaN,18_Novel,20.250257,9.656711,...,17.33726,5.2557,0.311958,35.72222,32.71796,0.668803,0.382134,9.656711,0.328205,1.783412


In [24]:
majorclass = "BC"
is_subtype_marker = np.logical_and(merged_filtered_markers["Major_Name"] == majorclass, merged_filtered_markers["Name"] != majorclass)
merged_filtered_markers.loc[is_subtype_marker, "Queried_Name"].drop_duplicates(keep='first').tolist()

['BC1B',
 'BC3A',
 'BC3B',
 'BC5A',
 'BC5B',
 'BC5C',
 'BC5D',
 'BC6',
 'BC7',
 'BC8',
 'RBC']

In [27]:
adata.obs.loc[adata.obs['majorclass'] == majorclass, "author_cell_type"].astype(str).sort_values().drop_duplicates(keep='first').tolist()

['BC1A',
 'BC1B',
 'BC2',
 'BC3A',
 'BC3B',
 'BC4',
 'BC5A',
 'BC5B',
 'BC5C',
 'BC5D',
 'BC6',
 'BC7',
 'BC8',
 'BC9',
 'RBC',
 'Unassigned_BC']

In [29]:
merged_filtered_markers

,Name,Marker,Curated,Queried,Minor_Coefficient,Major_Name,Major_Coefficient,Queried_Name,Raw_Mean_Expression_Subtype_Marker,Raw_Mean_Expression_Majorclass_Marker,...,BC,Cone,Endothelial,HC,MG,Microglia,Pericyte,RGC,RPE,Rod
0,AC1,Rbfox1,NaN,Queried,0.561532,AC,NaN,AC_1,9.738955,5.628321,...,0.082111,0.338376,0.02253,0.293651,0.076488,0.13856,0.091811,13.777289,0.005128,0.077222
1,AC1,Stmn1,NaN,Queried,0.414927,AC,NaN,AC_1,6.531861,4.484476,...,0.838333,0.11613,0.665511,1.34127,2.318807,0.82002,0.126551,21.688866,0.192308,0.079088
2,AC10,Kcnip4,NaN,Queried,0.479949,AC,NaN,AC_10,15.004365,7.037623,...,14.641668,0.02003,0.10052,1.690476,2.444016,0.165187,0.094293,43.91038,0.051282,0.037293
3,AC10,Nrxn1,NaN,Queried,0.339957,AC,NaN,AC_10,6.302778,4.514096,...,2.208109,0.083529,0.071057,9.753968,0.066204,0.049474,0.096774,8.582273,0.005128,0.014603
4,AC14,Frmd5,NaN,Queried,0.386736,AC,NaN,AC_14,9.316994,6.016793,...,0.477381,1.898146,1.745234,2.214286,1.613832,0.035996,0.114144,0.427972,0.023077,0.665936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,W3D2,Camk2d,NaN,Queried,0.392333,RGC,NaN,23_W3D2,8.792982,4.267854,...,3.394424,7.856169,1.07539,6.65873,3.024553,1.146121,1.141439,4.267854,0.15641,2.185782
227,W3D2,Kcnd2,NaN,Queried,0.911086,RGC,NaN,23_W3D2,30.659649,5.398889,...,0.520086,0.003835,0.019931,0.079365,0.025582,0.010848,0.079404,5.398889,0.0,0.003673
228,W3D2,Lrrc4c,NaN,Queried,0.515166,RGC,NaN,23_W3D2,17.440935,11.469259,...,4.938026,0.007245,0.215771,1.095238,3.650855,0.046351,1.042184,11.469259,0.007692,0.011996
229,W3D2,Pcdh7,NaN,Queried,0.797843,RGC,NaN,23_W3D2,27.031578,4.723438,...,3.198745,0.005753,0.227903,0.230159,0.933282,0.025148,0.007444,4.723438,0.064103,0.010278


In [30]:
adata.obs.loc[adata.obs['author_cell_type'].isin(merged_filtered_markers["Queried_Name"]), "author_cell_type"].astype(str).sort_values().drop_duplicates(keep='first').tolist()

['10_Novel',
 '12_ooDS_NT',
 '13_Novel',
 '14_ooDS_Cck',
 '15_Novel',
 '16_ooDS_DV',
 '17_Tbr1_S1',
 '18_Novel',
 '19_Novel',
 '1_W3D1.1',
 '20_Novel',
 '21_Tbr1_S2',
 '22_M5',
 '23_W3D2',
 '24_Novel',
 '25_Novel',
 '26_Novel',
 '27_Novel',
 '28_FmidiOFF',
 '29_Novel',
 '2_W3D1.2',
 '30_Novel',
 '31_M2',
 '33_M1',
 '34_Novel',
 '35_Novel',
 '36_Novel',
 '38_FmidiON',
 '3_FminiON',
 '41_AlphaONT',
 '44_Novel',
 '4_FminiOFF',
 '5_J-RGC',
 '6_W3B',
 '7_Novel',
 '8_Novel',
 '9_Tbr1_Novel',
 'AC_1',
 'AC_10',
 'AC_14',
 'AC_19',
 'AC_2',
 'AC_20',
 'AC_21',
 'AC_22',
 'AC_23',
 'AC_26',
 'AC_27',
 'AC_29',
 'AC_3',
 'AC_32',
 'AC_33',
 'AC_34',
 'AC_36',
 'AC_37',
 'AC_38',
 'AC_4',
 'AC_40',
 'AC_41',
 'AC_43',
 'AC_46',
 'AC_47',
 'AC_5',
 'AC_58',
 'AC_7',
 'AC_8',
 'AC_9',
 'BC1B',
 'BC3A',
 'BC3B',
 'BC5A',
 'BC5B',
 'BC5C',
 'BC5D',
 'BC6',
 'BC7',
 'BC8',
 'RBC']

In [34]:
merged_filtered_markers[["Name","Queried_Name"]].drop_duplicates()

,Name,Queried_Name
0,AC1,AC_1
2,AC10,AC_10
4,AC14,AC_14
5,AC19,AC_19
6,AC2,AC_2
...,...,...
207,TBR1-S2,21_Tbr1_S2
211,W3B,6_W3B
217,W3D1,1_W3D1.1
222,W3D1,2_W3D1.2


In [38]:
q2n = pd.read_csv('spreadsheets/queried_to_name.txt', sep ='\t', index_col=0)
q2n

,Queried_Name
Name,
AC,AC
AC1,AC_1
AC10,AC_10
AC11,AC_11
AC13,AC_13
...,...
W3D1,1_W3D1.1
W3D1,2_W3D1.2
W3D2,23_W3D2


In [46]:
a = ['AC_1', 'AC_10', 'AC_13', 'AC_18', 'AC_22', 'AC_42', 'AC_61', 'AC_9', 'BC2', 'BC3A', 'BC3B', 'BC5A', 'BC5B', 'BC6', 'BC9', 'RBC', '19_Novel', '20_Novel', '31_M2', '32_F_Novel', '40_M1dup', '41_AlphaONT', '42_AlphaOFFS', '43_AlphaONS', '7_Novel', '8_Novel', 'MG', 'RPE', 'ROD', 'CONE', 'HC', 'MICROGLIA', 'ENDOTHELIAL']
b = ['7_Novel', '8_Novel', '19_Novel', '20_Novel', '31_M2', '32_F_Novel', '40_M1dup', '41_AlphaONT', '42_AlphaOFFS', '43_AlphaONS', 'AC_1', 'AC_9', 'AC_10', 'AC_13', 'AC_18', 'AC_22', 'AC_42', 'AC_61', 'BC2', 'BC3A', 'BC3B', 'BC5A', 'BC5B', 'BC6', 'BC9', 'CONE', 'ENDOTHELIAL', 'HC', 'MG', 'RBC', 'ROD', 'RPE']
np.setdiff1d(a,b)

array(['MICROGLIA'], dtype='<U12')

In [45]:
adata.var["feature_name"].loc[adata.var["feature_name"].astype(str).str.contains("Cd74")]

ENSMUSG00000024610    Cd74
Name: feature_name, dtype: category
Categories (32034, object): ['0610005C13Rik', '0610006L08Rik', '0610009B22Rik', '0610009E02Rik', ..., 'mt-Nd4', 'mt-Nd4l', 'mt-Nd5', 'mt-Nd6']